In [1]:
!cat /proc/meminfo

MemTotal:       32949048 kB
MemFree:        24442480 kB
MemAvailable:   25913980 kB
Buffers:          680600 kB
Cached:           996392 kB
SwapCached:            0 kB
Active:          7505000 kB
Inactive:         621224 kB
Active(anon):    6449232 kB
Inactive(anon):       48 kB
Active(file):    1055768 kB
Inactive(file):   621176 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                80 kB
Writeback:             0 kB
AnonPages:       6449492 kB
Mapped:           156128 kB
Shmem:                60 kB
Slab:             283004 kB
SReclaimable:     258464 kB
SUnreclaim:        24540 kB
KernelStack:        5168 kB
PageTables:        19796 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    16474524 kB
Committed_AS:    6080684 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [2]:
!pip install xgboost

    100% |████████████████████████████████| 15.8MB 3.2MB/s eta 0:00:01
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import gc

from sklearn import preprocessing as prp
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import (accuracy_score, r2_score, confusion_matrix,
f1_score, recall_score, precision_score)

import os
import boto3
import re

import xgboost as xgb

# ruta de trabajo en s3
bucket = 'boi-banregio'
target = 'BMI'
feat_part_number = 40

# s3 resource
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

In [6]:
# Aquí colocamos la ruta de nuestro directorio, el nombre de nuestro archivo y si tiene o no cabecera
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION'

In [5]:
target_file_name = "TARGETS.csv"
target_file_key = "{}/{}".format(prefix, target_file_name)
target_obj = s3_bucket_resource.Object(target_file_key).get()
data_target = pd.read_csv(io.BytesIO(target_obj['Body'].read())).loc[:,[target]]

In [7]:
data_feature_list = []
for k in range(0, 10):
    data_file_name = "PROCESSED_DATA/DATA_PREPROCESSED_PART{}.csv".format(k)
    data_file_key = "{}/{}".format(prefix, data_file_name)
    print(data_file_key)
    
    feat_imp_file_name = "FEATURE_SELECTION/FEATURE_IMPORTANCE_PART{}.csv".format(k)
    feat_imp_file_key = "{}/{}".format(prefix, feat_imp_file_name)
    print(feat_imp_file_key)    
    
    print('Loading data')
    data_part_obj = s3_bucket_resource.Object(data_file_key).get()
    data_part = pd.read_csv(io.BytesIO(data_part_obj['Body'].read()))
    display(data_part)
    
    feat_imp_part_obj = s3_bucket_resource.Object(feat_imp_file_key).get()
    feat_imp_part = pd.read_csv(io.BytesIO(feat_imp_part_obj['Body'].read()))
    display(feat_imp_part)
    #break
    # features = feat_imp_part.loc[:feat_part_number-1, 'Feature'].values
    features = feat_imp_part['Feature'].values
    display(features)
    #break
    
    data_feat_imp_part = data_part[features]
    data_feature_list.append(data_feat_imp_part) 
    print('Step: ', k)

print('Uploading selected features data')
model_features = pd.concat(data_feature_list, axis=1)
model_features_file_name = "MODEL_DATASET/MODEL_FEATURES.csv"
model_features_obj_key =  "{}/{}".format(prefix, model_features_file_name)    

f_str = io.StringIO()
model_features.to_csv(f_str, index=False)
s3_bucket_resource.Object(model_features_obj_key).put(Body=f_str.getvalue())
print('Model features saved!')

datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/PROCESSED_DATA/DATA_PREPROCESSED_PART0.csv
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/FEATURE_SELECTION/FEATURE_IMPORTANCE_PART0.csv
Loading data


,AVG_PND_MENOS_BAL_12M,AVG_PND_MAS_BAL_12M,AVG_BAL_12M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_R_BAL_BY_CRED_LIM_12M,MIN_R_BAL_BY_MAX_BAL_03M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,BK12_MAX_CREDIT_AMT,...,PCT_BK_CS_CL_ACC,PCT_BK_CS_OP_ACC,PCT_BK_LSG_OP_ACC,PCT_BRG_CS_OP_ACC,PCT_GBN_CF_CL_ACC,PCT_GBN_PQ_CL_ACC,PCT_GBN_PQ_OP_ACC,PCT_LSE_CS_OP_ACC,PCT_UN_CRE_CF_CL_ACC,PCT_UN_CRE_CL_ACC
0,-0.369388,-0.369483,-0.375298,-0.144109,-0.105921,-1.743006,0.359358,0.439008,0.486416,-0.580645,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,-0.369265,-0.368961,-0.374872,-0.145578,-0.106867,-1.731119,0.359358,0.439008,0.486416,-0.548387,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,-0.369040,-0.368467,-0.374434,-0.145578,-0.107841,-1.718884,0.359358,0.439008,0.486416,-0.548387,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,-0.368852,-0.369006,-0.374798,-0.143087,-0.107402,-1.729053,-2.348602,0.109949,-0.127429,-0.612903,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,1.147149,1.089671,1.111538,0.201820,-0.230332,0.705472,0.326892,0.422180,0.468819,1.354839,...,0.736842,1.684211,0.000000,0.0,0.052632,0.0,0.0,0.0,0.0,0.0
5,1.143641,1.092350,1.111154,0.201820,-0.226706,0.704829,0.343199,0.413795,0.467903,1.354839,...,0.736842,1.684211,0.000000,0.0,0.052632,0.0,0.0,0.0,0.0,0.0
6,1.140501,1.096515,1.111703,-0.300884,-0.226406,0.705749,0.343199,0.405068,0.469374,1.225806,...,0.777778,1.333333,0.000000,0.0,0.055556,0.0,0.0,0.0,0.0,0.0
7,1.137357,1.098814,1.111308,-0.300884,-0.224886,0.705086,0.326359,0.403919,0.468427,1.225806,...,0.777778,1.333333,0.000000,0.0,0.055556,0.0,0.0,0.0,0.0,0.0
8,1.135393,1.101655,1.111770,-0.303925,-0.225506,0.705862,0.326359,0.406636,0.469677,1.225806,...,0.777778,1.333333,0.000000,0.0,0.055556,0.0,0.0,0.0,0.0,0.0
9,1.101933,0.994320,1.041039,0.201785,-0.038930,0.587296,-0.416527,0.372529,0.394162,1.419355,...,0.700000,1.200000,0.000000,0.0,0.050000,0.0,0.0,0.0,0.0,0.0


,Feature,Importance
0,AVG_MAX_DIAS_PARA_PAGO_U06M,0.095238
1,AVG2_VP_MES_2,0.085714
2,AVG_R_BAL_BY_CRED_LIM_12M,0.047619
3,DEUDA_TOT,0.047619
4,MIN_R_BAL_BY_MAX_BAL_03M,0.047619
5,MAX_VENC1M_BRG_OP_ACC_06M,0.047619
6,AVG_AVG2_VP_MES_U03M_2,0.038095
7,AVG_VENC1M_BK_OP_ACC,0.028571
8,MIN_AVG2_VP_PER_U03M_2,0.028571
9,NUM_VIGENTEH_12M,0.028571


array(['AVG_MAX_DIAS_PARA_PAGO_U06M', 'AVG2_VP_MES_2',
       'AVG_R_BAL_BY_CRED_LIM_12M', 'DEUDA_TOT',
       'MIN_R_BAL_BY_MAX_BAL_03M', 'MAX_VENC1M_BRG_OP_ACC_06M',
       'AVG_AVG2_VP_MES_U03M_2', 'AVG_VENC1M_BK_OP_ACC',
       'MIN_AVG2_VP_PER_U03M_2', 'NUM_VIGENTEH_12M',
       'AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M', 'AVG_VIGENTEC_BRG_OP_ACC',
       'MAX_SALINI_OTHER_FIN_R_OP_ACC', 'MAX_AVG2_VP_MES_U06M',
       'AVG_PCT_VIGENTEC_BRG_OP_ACC_06M', 'AVG_VENC30M_BRG_CS_OP_ACC_03M',
       'MAX_MAX_HI_CALCAR_FLOOR_06M',
       'AVG_PND_MENOS_AVG_HI_CALCAR_FLOOR_06M',
       'MAX_PCT_VENC29_OP_ACC_12M', 'AVG_PCT_VIGENTEC_BRG_CS_OP_ACC_06M',
       'AVG_AVG_HI_CALCAR_CEIL_12M', 'MAX_VENC29_OP_ACC_03M',
       'AVG_AVG1_VP_PER_U03M_2', 'MIN_SALINI_LSE_OP_ACC_03M',
       'AVG_VIGENTEC_OTHER_FIN_OP_ACC_12M',
       'MAX_VIGENTEC_BRG_R_OP_ACC_03M', 'MIN_AVG1_VP_MES_U03M',
       'AVG_VIGENTEC_BK_CS_OP_ACC_03M', 'MIN_MIN_VP_MES_U12M',
       'AVG_VIGENTEC_LSE_LSG_OP_ACC_12M', 'AVG_SALINI_CL

Step:  0
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/PROCESSED_DATA/DATA_PREPROCESSED_PART1.csv
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/FEATURE_SELECTION/FEATURE_IMPORTANCE_PART1.csv
Loading data


...

Step:  8
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/PROCESSED_DATA/DATA_PREPROCESSED_PART9.csv
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/FEATURE_SELECTION/FEATURE_IMPORTANCE_PART9.csv
Loading data


,MIN_BAL_06M,MIN_BAL_12M,AVG_PND_MENOS_CRED_LIM_03M,AVG_CRED_LIM_06M,AVG_D_CRED_LIM_TO_BAL_03M,R_BAL_BY_CRED_LIM,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_03M,AVG_R_BAL_BY_MAX_BAL_03M,BK12_NUM_EXP_PAIDONTIME,NBK12_COMM_PCT_PLUS,...,NUM_LSE_CF_CL_ACC,NUM_LSE_CS_OP_ACC,NUM_LSE_LSG_OP_ACC,NUM_OTHER_FIN_R_CL_ACC,NUM_UN_CRE_LSG_CL_ACC,NUM_UN_CRE_R_OP_ACC,PCT_BK_OP_ACC,PCT_BRG_LSE_OTHER_CL_ACC,PCT_BRG_OTHER_CL_ACC,PCT_LSE_CF_CL_ACC
0,-0.315335,-0.297711,-0.339623,-0.339623,-0.098578,-1.453156,0.406303,0.406312,0.00,-0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.444444,0.0,0.0,0.0
1,-0.315335,-0.297711,-0.339623,-0.339623,-0.100443,-1.433450,0.406303,0.406312,0.00,-0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.444444,0.0,0.0,0.0
2,-0.315335,-0.297711,-0.339623,-0.339623,-0.101417,-1.412878,0.406303,0.406312,0.00,-0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.444444,0.0,0.0,0.0
3,-0.316069,-0.298618,-0.339623,-0.339623,-0.099835,-1.466877,-0.604682,-1.978982,0.00,-0.50,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.444444,0.0,0.0,0.0
4,1.184113,1.555460,0.547170,0.547170,-0.143234,0.596788,0.366816,0.377155,0.00,2.00,...,0.0,0.0,0.0,1.0,0.0,0.0,1.029240,0.0,0.0,0.0
5,1.184113,1.555460,0.547170,0.547170,-0.143801,0.602741,0.393761,0.393334,0.00,2.00,...,0.0,0.0,0.0,1.0,0.0,0.0,1.029240,0.0,0.0,0.0
6,1.184113,1.555460,0.547170,0.547170,-0.311328,0.616805,0.394653,0.396578,0.00,2.25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.888889,0.0,0.0,0.0
7,1.184113,1.555460,0.547170,0.547170,-0.308920,0.599237,0.385326,0.382497,0.00,2.25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.888889,0.0,0.0,0.0
8,1.184113,1.555460,0.547170,0.547170,-0.311771,0.604017,0.385830,0.379035,0.00,2.25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.888889,0.0,0.0,0.0
9,1.184138,1.555460,0.547170,0.547170,0.177540,0.193444,0.080766,-0.298280,0.00,2.25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.955556,0.0,0.0,0.0


,Feature,Importance
0,AVG_MAX_DIAS_PARA_PAGO_U03M,0.057143
1,AVG_R_BAL_BY_MAX_BAL_03M,0.057143
2,AVG1_VP_MES_2,0.057143
3,R_BAL_BY_CRED_LIM,0.057143
4,MIN_VIGENTEH_12M,0.038095
5,AVG_PND_MAS_PCT_VENC89_03M,0.028571
6,SUM_PAGOREALIZADO_PERIODO,0.028571
7,MAX_PCT_VENC29_BRG_OP_ACC_03M,0.028571
8,AVG_PND_MAS_MAX_HI_CALCAR_L_12M,0.028571
9,AVG_PCT_VIGENTEC_BRG_R_OP_ACC_06M,0.028571


array(['AVG_MAX_DIAS_PARA_PAGO_U03M', 'AVG_R_BAL_BY_MAX_BAL_03M',
       'AVG1_VP_MES_2', 'R_BAL_BY_CRED_LIM', 'MIN_VIGENTEH_12M',
       'AVG_PND_MAS_PCT_VENC89_03M', 'SUM_PAGOREALIZADO_PERIODO',
       'MAX_PCT_VENC29_BRG_OP_ACC_03M', 'AVG_PND_MAS_MAX_HI_CALCAR_L_12M',
       'AVG_PCT_VIGENTEC_BRG_R_OP_ACC_06M', 'MAX_PCT_VENC30M_12M',
       'MAX_VENC29_BK_CS_OP_ACC_03M', 'MAX_VIGENTEC_LSE_OP_ACC_03M',
       'AVG_MS_OP_BRG_CS_OP_ACC', 'MAX_AVG2_VP_MES_U06M_2',
       'MAX_PCT_VENC1M_BK_OP_ACC_03M', 'MIN_AVG1_VP_PER_U12M_2',
       'AVG_PND_MAS_MAX_HI_CALCAR_CEIL_06M',
       'MAX_MAX_DIAS_PARA_PAGO_U03M', 'MAX_VENC1M_OP_ACC_12M',
       'AVG1_VP_PER_2', 'MIN_BAL_12M', 'AVG_AVG2_VP_MES_U03M',
       'AVG2_VP_MES', 'MAX_MAX_VP_PER_U12M', 'MIN_SALINI_CL_ACC_06M',
       'MAX_MS_CL_BRG_CS_CL_ACC', 'AVG_MS_OP_OTHER_FIN_LSG_OP_ACC',
       'MAX_SALINI_BK_CL_ACC_12M', 'AVG_VIGENTEC_OP_ACC_03M',
       'MAX_VIGENTEC_BRG_OP_ACC_03M', 'AVG_AVG_HI_CALCAR_L_06M',
       'NBK12_COMM_PCT_PLUS', '

Step:  9
Uploading selected features data
Model features saved!


In [8]:
display(model_features.describe())
display(model_features)

,AVG_MAX_DIAS_PARA_PAGO_U06M,AVG2_VP_MES_2,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_VENC1M_BRG_OP_ACC_06M,AVG_AVG2_VP_MES_U03M_2,AVG_VENC1M_BK_OP_ACC,MIN_AVG2_VP_PER_U03M_2,NUM_VIGENTEH_12M,...,AVG_MIN_VP_PER_U12M_2,AVG_PND_MAS_VENC59_06M,MIN_SALINI_CL_ACC_03M,AVG_PCT_VIGENTEC_OP_ACC,MAX_PCT_VENC1M_BRG_CS_OP_ACC_06M,MIN_AVG_HI_CALCAR_CEIL_12M,MAX_VIGENTEC_BK_CS_OP_ACC_03M,AVG_MAX_VP_MES_U06M,AVG_PCT_VENC1M_BK_R_OP_ACC_12M,MAX_SALINI_LSE_CL_ACC
count,425287.000000,4.252870e+05,425287.000000,425287.000000,425287.000000,4.252870e+05,4.252870e+05,4.252870e+05,425287.000000,425287.000000,...,4.252870e+05,425287.000000,4.252870e+05,425287.000000,425287.000000,425287.000000,4.252870e+05,4.252870e+05,425287.000000,4.252870e+05
mean,2.067763,9.503875e+02,0.012022,3.243717,-0.483808,6.531460e+03,3.362386e+02,2.013845e+04,1.863932,-2.298373,...,4.625206e+01,9.865940,1.859141e+05,-0.647893,0.006982,0.855755,5.575071e+05,3.438702e+02,0.015172,1.083993e+05
std,4.956463,4.281314e+05,23.915491,350.995352,1.100280,9.721396e+04,8.409633e+04,2.783602e+05,306.968965,4.059971,...,4.062621e+03,129.429375,3.057280e+06,1.037909,0.062519,6.743288,1.655065e+07,7.172509e+04,0.103105,1.574319e+06
min,0.000000,-5.525721e-01,-1.789694,-0.663866,-3.601348,0.000000e+00,-1.561660e-01,0.000000e+00,-1.101120,-12.000000,...,-3.548131e+00,0.000000,-1.900000e+05,-4.988842,0.000000,-0.944444,-1.012809e+06,-1.457683e-01,0.000000,-2.738500e+05
25%,0.000000,0.000000e+00,-0.576496,-0.386555,-0.733688,0.000000e+00,-1.154868e-04,0.000000e+00,-1.000000,-1.000000,...,-9.764420e-01,0.000000,0.000000e+00,-0.988842,0.000000,-0.944444,0.000000e+00,-1.838367e-02,0.000000,0.000000e+00
50%,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
75%,1.000000,1.000000e+00,0.423504,0.613445,0.266312,0.000000e+00,9.998845e-01,0.000000e+00,0.000000,0.000000,...,2.355804e-02,0.000000,0.000000e+00,0.011158,0.000000,0.055556,0.000000e+00,9.816163e-01,0.000000,0.000000e+00
max,24.889165,2.762861e+08,7368.944220,56249.680672,0.359358,6.880380e+06,3.901264e+07,4.026163e+07,139577.444070,0.000000,...,1.422125e+06,25321.714286,4.613100e+08,0.011158,1.000000,164.222222,3.501605e+09,3.184616e+07,1.000000,1.464574e+08


,AVG_MAX_DIAS_PARA_PAGO_U06M,AVG2_VP_MES_2,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_VENC1M_BRG_OP_ACC_06M,AVG_AVG2_VP_MES_U03M_2,AVG_VENC1M_BK_OP_ACC,MIN_AVG2_VP_PER_U03M_2,NUM_VIGENTEH_12M,...,AVG_MIN_VP_PER_U12M_2,AVG_PND_MAS_VENC59_06M,MIN_SALINI_CL_ACC_03M,AVG_PCT_VIGENTEC_OP_ACC,MAX_PCT_VENC1M_BRG_CS_OP_ACC_06M,MIN_AVG_HI_CALCAR_CEIL_12M,MAX_VIGENTEC_BK_CS_OP_ACC_03M,AVG_MAX_VP_MES_U06M,AVG_PCT_VENC1M_BK_R_OP_ACC_12M,MAX_SALINI_LSE_CL_ACC
0,0.000000,0.000000,-1.743006,-0.025210,0.359358,0.0,0.000000,0.000000e+00,0.000000,-12.0,...,0.000000,0.000000,0.0,-2.488842,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,0.000000,0.000000,-1.731119,0.016807,0.359358,0.0,0.000000,0.000000e+00,0.000000,-11.0,...,0.000000,0.000000,0.0,-2.488842,0.0,0.055556,0.0,0.000000,0.000000,0.0
2,0.000000,0.000000,-1.718884,0.016807,0.359358,0.0,0.000000,0.000000e+00,0.000000,-11.0,...,0.000000,0.000000,0.0,-2.488842,0.0,0.055556,0.0,0.000000,0.000000,0.0
3,0.000000,0.000000,-1.729053,0.058824,-2.348602,0.0,0.000000,0.000000e+00,0.000000,-9.0,...,0.000000,0.000000,0.0,-4.988842,0.0,0.055556,0.0,0.000000,0.000000,0.0
4,0.000000,0.835695,0.705472,-0.142857,0.326892,18723.0,1.535494,0.000000e+00,0.000000,0.0,...,-1.206465,0.714286,1310000.0,-1.356694,0.0,1.222222,1997576.0,0.701375,0.000000,33684.0
5,0.000000,-0.184918,0.704829,-0.142857,0.343199,18723.0,1.508949,0.000000e+00,0.000000,0.0,...,-1.206465,0.714286,1310000.0,-1.356694,0.0,1.222222,1997576.0,0.696731,0.000000,33684.0
6,0.000000,1.488142,0.705749,-0.117647,0.343199,0.0,0.513641,0.000000e+00,0.000000,0.0,...,-1.206465,0.000000,2010000.0,-0.488842,0.0,1.013889,1998241.0,0.749407,0.000000,33684.0
7,0.000000,0.408234,0.705086,-0.117647,0.326359,0.0,0.179718,0.000000e+00,0.000000,0.0,...,-1.206465,0.000000,2010000.0,-0.488842,0.0,1.013889,1998241.0,0.754279,0.000000,33684.0
8,0.000000,-0.081950,0.705862,-0.117647,0.326359,0.0,0.114931,0.000000e+00,0.000000,0.0,...,-1.206465,0.000000,2010000.0,-0.488842,0.0,1.013889,1998241.0,0.745288,0.000000,33684.0
9,0.000000,0.094818,0.587296,-0.092437,-0.416527,0.0,0.108030,0.000000e+00,0.000000,0.0,...,0.000000,0.000000,-38000.0,-0.443387,0.0,1.013889,2000191.0,0.088425,0.000000,33684.0


In [36]:
feat_imp_file_name = "DATA_PART{}.csv".format(5)
feat_imp_file_key = "{}/{}".format(prefix, feat_imp_file_name)
feat_imp_part_obj = s3_bucket_resource.Object(feat_imp_file_key).get()
feat_imp_part = pd.read_csv(io.BytesIO(feat_imp_part_obj['Body'].read()))

In [38]:
feat_imp_part.loc[:,'REVOLVENTE'].describe()

count    425287.000000
mean          0.766080
std           0.423322
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: REVOLVENTE, dtype: float64